In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn import ensemble
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
import numpy as np

In [3]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll.base import scope

In [4]:
#!pip install pycaret==2.0

In [5]:
train_path = "/content/drive/MyDrive/Colab Notebooks/ds_projects/train.csv"

In [6]:
train_data = pd.read_csv(train_path)

In [7]:
train_data.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [8]:
data_without_category = train_data.select_dtypes(include=["float64", "int64"])

In [9]:
X = data_without_category.loc[:, data_without_category.columns != "SalePrice"]
y = data_without_category["SalePrice"]

In [10]:
imputer = SimpleImputer(strategy = "mean", missing_values = np.nan)
X_imputed = imputer.fit_transform(X)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)

In [12]:
X_train_imputed, X_val_imputed, y_train_imputed, y_val_imputed = train_test_split(X_imputed, y, test_size=0.10, random_state=42)

In [13]:
print(X_train.shape, X_val.shape)

(1314, 37) (146, 37)


## PyCart AutoML Test

In [ ]:
from pycaret.regression import *
from pycaret.utils import version

In [ ]:
version()

2.0


In [ ]:
model_automl = setup( data = data_without_category, target = 'SalePrice', normalize = True,
                   transform_target = False, remove_outliers= True,
                   remove_multicollinearity = True,
                   ignore_low_variance = True, combine_rare_levels = True, folds_shuffle=True, data_split_shuffle=True)

 
Setup Succesfully Completed.


,Description,Value
0,session_id,5009
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(1460, 38)"
4,Missing Values,True
5,Numeric Features,21
6,Categorical Features,16
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 0
INFO:logs:setup() succesfully completed......................................


In [ ]:
#print(setup.__doc__)

In [ ]:
gbest = compare_models(blacklist = ['tr'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,CatBoost Regressor,15816.1636,587587228.0542,23910.5741,0.8845,0.1304,0.0942,4.9788
1,Ridge Regression,16607.6109,574524009.8760,23709.6249,0.8836,0.1368,0.0993,0.0237
2,Huber Regressor,16055.5025,592159311.4797,24030.2615,0.8834,0.1300,0.0940,0.1249
3,Bayesian Ridge,16654.9404,579649071.3742,23822.1856,0.8833,0.1368,0.0998,0.0533
4,Lasso Regression,16683.8840,581082260.5725,23823.4130,0.8816,0.1377,0.0996,0.2336
5,Lasso Least Angle Regression,16684.6080,581199608.9124,23825.6365,0.8815,0.1377,0.0996,0.0568
6,Linear Regression,16695.1541,581653238.6920,23834.7998,0.8814,0.1378,0.0997,0.0525
7,Random Sample Consensus,16498.7765,598651124.7283,24243.6730,0.8788,0.1364,0.0977,1.4099
8,Gradient Boosting Regressor,16876.4568,665532123.1414,25432.3858,0.8680,0.1383,0.1004,0.6336
9,Light Gradient Boosting Machine,17644.8915,712397732.5537,26362.8355,0.8563,0.1444,0.1049,0.1736


INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:<catboost.core.CatBoostRegressor object at 0x7fa7ce2d54e0>
INFO:logs:compare_models() succesfully completed......................................


In [ ]:
predictions = predict_model(gbest, X_val)

In [ ]:
predictions

0      145236.2016
1      320693.2719
2      113118.5960
3      155160.2310
4      311176.9582
          ...     
141    136165.0401
142    174938.5681
143    148684.5713
144    143249.2588
145    132300.6363
Name: Label, Length: 146, dtype: float64

In [ ]:
mean_squared_error(y_val, predictions["Label"], squared=False)

34050.10741808393

In [ ]:
mean_absolute_error(y_val, predictions["Label"])

10985.168515068495

## Sklearn + Hyperparam Grid Search + Cross Validation + XGboost




In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# Define the parameter grid for tuning
param_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5],
    'n_estimators': [50, 100],
    'subsample': [0.8],
    'colsample_bytree': [0.8]
}

# Initialize the XGBoost regressor
xgb = XGBRegressor(objective='reg:squarederror')

# Perform grid search with cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Define your desired number of folds
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring='neg_mean_squared_error', cv=kf)

# Fit the grid search to the data
grid_search.fit(X, y, verbose=True)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Print the best parameters
print("Best Parameters:", best_params)

# Calculate and print the mean squared error (MSE) for the best model
y_pred = best_model.predict(X)
mse = mean_squared_error(y, y_pred)
print("Mean Squared Error:", mse)


Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.8}
Mean Squared Error: 182941105.48443502


In [ ]:
xgboost_model =XGBRegressor(**best_params)

In [ ]:
xgboost_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=50, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
mean_squared_error(y_val, xgboost_model.predict(X_val), squared = False) #rmse

33185.88524842642

In [ ]:
mean_squared_error(y_train, xgboost_model.predict(X_train), squared = False) #rmse

13897.77303617247

## Baysean Hyperparameter Search + Cross Validation + XGboost

In [14]:
space = {
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'n_estimators': hp.choice('n_estimators', [10, 20, 30]),
    'subsample': hp.uniform('subsample', 0.8, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.8, 1.0)
}

In [21]:
def objective(params):
    # Create an XGBoost regressor with the given hyperparameters
    xgb = XGBRegressor(
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        n_estimators=params['n_estimators'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree']
    )

    repeated_cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=1) #not ideal params -> adjust for real-world application

    # Perform cross-validation with mean squared error as the scoring metric
    mse_scores = -cross_val_score(xgb, X, y, cv=repeated_cv, scoring='neg_mean_squared_error')

    ## Hyperopt is used for minimizing objective and cross validation is maximizing

    # Return the mean of the MSE scores as the objective value
    return np.mean(mse_scores)

In [22]:
trials = Trials()

In [23]:
best_model = fmin(
    fn=objective, space=space, algo = tpe.suggest, max_evals = 100, trials = trials
)

100%|██████████| 100/100 [00:48<00:00,  2.07trial/s, best loss: 913558880.1467091]


In [ ]:
print(best_model)

{'colsample_bytree': 0.8431340549072125, 'learning_rate': 0.19878470909797186, 'max_depth': 3, 'n_estimators': 2, 'subsample': 0.9936295527522215}
